In [ ]:
from lissi_realsense import MyReader
import cv2
import opencv_jupyter_ui as jcv2
import lissilab
import random
import shutil
import os

pathToBag = '/raid/home/labuseraber/environment/testEnvironment/dmo08082022/dmo/dmoFake2/'
pathToBag = '/raid/home/labuseraber/environment/testEnvironment/dmo08082022/dmo/dmoFake/'
pathToBag = '/raid/home/labuseraber/environment/testEnvironment/dmo08082022/dmo/dmoReal/'

saveRealDepth = './realDepth.jpg'

reader = MyReader(pathToBag)

cap = cv2.VideoCapture(pathToBag + 'Color.avi')
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

randomlist = random.sample(range(length), 10)

real = 'real'
fake = 'fake'

realList = []
fakeList = []

for i in randomlist:
    reader._read_frame(i)
    color=reader.get_color_frame()
    depth=reader.get_depth_frame()
    infrared=reader.get_infrared_frame(1)
    depth_color=reader.get_colorized_depth()

    face_landmarks_list = lissilab.face_landmarks(color)

    chin01 = face_landmarks_list[0]['chin'][1]
    chin15 = face_landmarks_list[0]['chin'][15]
    nose_tip = face_landmarks_list[0]['nose_tip'][2]
    left_eye = face_landmarks_list[0]['left_eye'][5]
    right_eye = face_landmarks_list[0]['right_eye'][5]

    chinDistance01 = reader.measure_depth(chin01[0], chin01[1])
    chinDistance01 = "{:.2f}".format(chinDistance01)
    chinDistance15 = reader.measure_depth(chin15[0], chin15[1])
    chinDistance15 = "{:.2f}".format(chinDistance15)
    nose_tipDistance = reader.measure_depth(nose_tip[0], nose_tip[1])
    nose_tipDistance = "{:.2f}".format(nose_tipDistance)
    left_eyeDistance = reader.measure_depth(left_eye[0], left_eye[1])
    left_eyeDistance = "{:.2f}".format(left_eyeDistance)
    right_eyeDistance = reader.measure_depth(right_eye[0], right_eye[1])
    right_eyeDistance = "{:.2f}".format(right_eyeDistance)

    if chinDistance01 != left_eyeDistance and \
        chinDistance01 != nose_tipDistance and \
        chinDistance01 != right_eyeDistance and \
        chinDistance15 != left_eyeDistance and \
        chinDistance15 != nose_tipDistance and \
        chinDistance15 != right_eyeDistance and \
        nose_tipDistance != right_eyeDistance and \
        nose_tipDistance != left_eyeDistance:
        realList.append(real)
        realImage = color
        face_location = lissilab.face_locations(realImage)
        x, y, w, h = face_location[0]
        realImage = realImage[x-20:w, h:y]

    elif chinDistance01 == left_eyeDistance and \
        chinDistance01 == nose_tipDistance and \
        chinDistance01 == right_eyeDistance and \
        chinDistance15 == left_eyeDistance and \
        chinDistance15 == nose_tipDistance and \
        chinDistance15 == right_eyeDistance and \
        nose_tipDistance == right_eyeDistance and \
        nose_tipDistance == left_eyeDistance:
        fakeList.append(fake)

realDict = {i:realList.count(i) for i in realList}
fakeDict = {i:fakeList.count(i) for i in fakeList}

if realList and fakeList:
    if realDict[real] > fakeDict[fake]:
        print(real)
        cv2.imwrite(saveRealDepth, realImage)
    elif fakeDict[fake] > realDict[real]:
        print(fake)
    elif fakeDict[fake] == realDict[real]:
        print('Do another capture')
elif realList and not fakeList:
    print(real)
    cv2.imwrite(saveRealDepth, realImage)
elif fakeList and not realList:
    print(fake)

shutil.rmtree(pathToBag + 'Color')
shutil.rmtree(pathToBag + 'Depth')
os.remove(pathToBag + 'Color.avi')
os.remove(pathToBag + 'Depth.avi')
os.remove(pathToBag + 'meta.pkl')

In [ ]:
# Decode a DPO
# Process BAG file and returns .jpg if real face

from lissi_realsense import MyReader
import cv2
import opencv_jupyter_ui as jcv2
import lissilab
import random
import shutil
import json
import os

pathToJSON = './dmoDPOTest.json'
pathToDPO  = './' # Do not forget /
pathToBAG  = './'
pathToAndNameOfOut = './realDepth.jpg'

r = open(pathToJSON)
y = json.load(r)

ext = y[0]['iai_dpo_metadata'][0]['file:extension']

if ext == 'bag':
    bagName = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    print('\nBAGName: ', bagName)
    statusBAG = os.system("base64 -d " + pathToDPO + bagName + "> " + pathToDPO + "a.bag")

    reader = MyReader(pathToBAG)

    cap = cv2.VideoCapture(pathToBAG + 'Color.avi')
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    randomlist = random.sample(range(length), 10)

    real = 'real'
    fake = 'fake'

    realList = []
    fakeList = []

    for i in randomlist:
        reader._read_frame(i)
        color=reader.get_color_frame()
        depth=reader.get_depth_frame()
        infrared=reader.get_infrared_frame(1)
        depth_color=reader.get_colorized_depth()

        face_landmarks_list = lissilab.face_landmarks(color)

        chin01 = face_landmarks_list[0]['chin'][1]
        chin15 = face_landmarks_list[0]['chin'][15]
        nose_tip = face_landmarks_list[0]['nose_tip'][2]
        left_eye = face_landmarks_list[0]['left_eye'][5]
        right_eye = face_landmarks_list[0]['right_eye'][5]

        chinDistance01 = reader.measure_depth(chin01[0], chin01[1])
        chinDistance01 = "{:.2f}".format(chinDistance01)
        chinDistance15 = reader.measure_depth(chin15[0], chin15[1])
        chinDistance15 = "{:.2f}".format(chinDistance15)
        nose_tipDistance = reader.measure_depth(nose_tip[0], nose_tip[1])
        nose_tipDistance = "{:.2f}".format(nose_tipDistance)
        left_eyeDistance = reader.measure_depth(left_eye[0], left_eye[1])
        left_eyeDistance = "{:.2f}".format(left_eyeDistance)
        right_eyeDistance = reader.measure_depth(right_eye[0], right_eye[1])
        right_eyeDistance = "{:.2f}".format(right_eyeDistance)

        if chinDistance01 != left_eyeDistance and \
            chinDistance01 != nose_tipDistance and \
            chinDistance01 != right_eyeDistance and \
            chinDistance15 != left_eyeDistance and \
            chinDistance15 != nose_tipDistance and \
            chinDistance15 != right_eyeDistance and \
            nose_tipDistance != right_eyeDistance and \
            nose_tipDistance != left_eyeDistance:
            realList.append(real)
            realImage = color
            face_location = lissilab.face_locations(realImage)
            x, y, w, h = face_location[0]
            realImage = realImage[x-20:w, h:y]

        elif chinDistance01 == left_eyeDistance and \
            chinDistance01 == nose_tipDistance and \
            chinDistance01 == right_eyeDistance and \
            chinDistance15 == left_eyeDistance and \
            chinDistance15 == nose_tipDistance and \
            chinDistance15 == right_eyeDistance and \
            nose_tipDistance == right_eyeDistance and \
            nose_tipDistance == left_eyeDistance:
            fakeList.append(fake)

    realDict = {i:realList.count(i) for i in realList}
    fakeDict = {i:fakeList.count(i) for i in fakeList}

    if realList and fakeList:
        if realDict[real] > fakeDict[fake]:
            print(real)
            cv2.imwrite(pathToAndNameOfOut, realImage)
        elif fakeDict[fake] > realDict[real]:
            print(fake)
        elif fakeDict[fake] == realDict[real]:
            print('Do another capture')
    elif realList and not fakeList:
        print(real)
        cv2.imwrite(pathToAndNameOfOut, realImage)
    elif fakeList and not realList:
        print(fake)
    
    shutil.rmtree(pathToBAG + 'Color')
    shutil.rmtree(pathToBAG + 'Depth')
    os.remove(pathToBAG + 'Color.avi')
    os.remove(pathToBAG + 'Depth.avi')
    os.remove(pathToBAG + 'meta.pkl')
    os.remove(pathToBAG + 'a.bag')

else:
    print("Check the DPO JSON - There is no .bag extension")

In [1]:
from lissi_realsense import MyReader
import cv2
import opencv_jupyter_ui as jcv2
import lissilab
import random
import shutil
import json
import os
from flask import Flask, jsonify, request

app = Flask(__name__)
DEBUG = ('DEBUG' in os.environ and os.environ['DEBUG'] in ['1', 'true'])

@app.route('/')    
def home():
    return '<h1>.--- . - .----. .- .. -- . .-.. -.-- -.-. .. .-</h1>'

@app.route("/dmo_process", methods = ['GET', 'POST'])
def dmo():
    # payload = request.json

    # dmofile = payload['dmo_file']
    # dmoparams = getattr(payload, 'dmo_params', None)

    # app.logger.info('New request: {}'.format(dmofile))

    try:
        pathToJSON = './dmoDPOTest.json'
        pathToDPO  = './' # Do not forget /
        pathToBAG  = './'
        pathToAndNameOfOut = './realDepth.jpg'

        r = open(pathToJSON)
        y = json.load(r)

        ext = y[0]['iai_dpo_metadata'][0]['file:extension']

        if ext == 'bag':
            bagName = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
            statusBAG = os.system("base64 -d " + pathToDPO + bagName + "> " + pathToBAG + "a.bag")

            reader = MyReader(pathToBAG)
            
            cap = cv2.VideoCapture(pathToBAG + 'Color.avi')
            length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            randomlist = random.sample(range(length), 10)

            real = 'real'
            fake = 'fake'

            realList = []
            fakeList = []

            for i in randomlist:
                reader._read_frame(i)
                color=reader.get_color_frame()
                depth=reader.get_depth_frame()
                infrared=reader.get_infrared_frame(1)
                depth_color=reader.get_colorized_depth()

                face_landmarks_list = lissilab.face_landmarks(color)

                chin01 = face_landmarks_list[0]['chin'][1]
                chin15 = face_landmarks_list[0]['chin'][15]
                nose_tip = face_landmarks_list[0]['nose_tip'][2]
                left_eye = face_landmarks_list[0]['left_eye'][5]
                right_eye = face_landmarks_list[0]['right_eye'][5]

                chinDistance01 = reader.measure_depth(chin01[0], chin01[1])
                chinDistance01 = "{:.2f}".format(chinDistance01)
                chinDistance15 = reader.measure_depth(chin15[0], chin15[1])
                chinDistance15 = "{:.2f}".format(chinDistance15)
                nose_tipDistance = reader.measure_depth(nose_tip[0], nose_tip[1])
                nose_tipDistance = "{:.2f}".format(nose_tipDistance)
                left_eyeDistance = reader.measure_depth(left_eye[0], left_eye[1])
                left_eyeDistance = "{:.2f}".format(left_eyeDistance)
                right_eyeDistance = reader.measure_depth(right_eye[0], right_eye[1])
                right_eyeDistance = "{:.2f}".format(right_eyeDistance)

                if chinDistance01 != left_eyeDistance and \
                    chinDistance01 != nose_tipDistance and \
                    chinDistance01 != right_eyeDistance and \
                    chinDistance15 != left_eyeDistance and \
                    chinDistance15 != nose_tipDistance and \
                    chinDistance15 != right_eyeDistance and \
                    nose_tipDistance != right_eyeDistance and \
                    nose_tipDistance != left_eyeDistance:
                    realList.append(real)
                    realImage = color
                    face_location = lissilab.face_locations(realImage)
                    x, y, w, h = face_location[0]
                    realImage = realImage[x-20:w, h:y]

                elif chinDistance01 == left_eyeDistance and \
                    chinDistance01 == nose_tipDistance and \
                    chinDistance01 == right_eyeDistance and \
                    chinDistance15 == left_eyeDistance and \
                    chinDistance15 == nose_tipDistance and \
                    chinDistance15 == right_eyeDistance and \
                    nose_tipDistance == right_eyeDistance and \
                    nose_tipDistance == left_eyeDistance:
                    fakeList.append(fake)

            realDict = {i:realList.count(i) for i in realList}
            fakeDict = {i:fakeList.count(i) for i in fakeList}

            if realList and fakeList:
                if realDict[real] > fakeDict[fake]:
                    print(real)
                    cv2.imwrite(pathToAndNameOfOut, realImage)
                elif fakeDict[fake] > realDict[real]:
                    print(fake)
                elif fakeDict[fake] == realDict[real]:
                    print('Do another capture')
            elif realList and not fakeList:
                print(real)
                cv2.imwrite(pathToAndNameOfOut, realImage)
            elif fakeList and not realList:
                print(fake)
            
            shutil.rmtree(pathToBAG + 'Color')
            shutil.rmtree(pathToBAG + 'Depth')
            os.remove(pathToBAG + 'Color.avi')
            os.remove(pathToBAG + 'Depth.avi')
            os.remove(pathToBAG + 'meta.pkl')
            os.remove(pathToBAG + 'a.bag')

        else:
            print("Check the DPO JSON - There is no .bag extension")

        return('', 204)
    except:
        return(jsonify({'error': 'Error occurred'}), 500)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=50000, debug=DEBUG)

In [ ]:
from lissi_realsense import MyReader
import cv2
import opencv_jupyter_ui as jcv2
import lissilab
import random
import shutil
import json
import os
import time

pathToJSON = './dmoDPOTest.json'
pathToDPO  = './' # Do not forget /
pathToBAG  = './' # Do not forget /
pathToAndNameOfOut = './realDepth.jpg'

r = open(pathToJSON)
y = json.load(r)

ext = y[0]['iai_dpo_metadata'][0]['file:extension']

if ext == 'bag':
    bagName = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    print('\nBAGName: ', bagName)
    statusBAG = os.system("base64 -d " + pathToDPO + bagName + "> " + pathToBAG + "a.bag")
    time.sleep(2)

    reader = MyReader(pathToBAG)

    cap = cv2.VideoCapture(pathToBAG + 'Color.avi')
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    randomlist = random.sample(range(length), 10)

    real = 'real'
    fake = 'fake'

    realList = []
    fakeList = []

    for i in randomlist:
        reader._read_frame(i)
        color=reader.get_color_frame()
        depth=reader.get_depth_frame()
        infrared=reader.get_infrared_frame(1)
        depth_color=reader.get_colorized_depth()

        face_landmarks_list = lissilab.face_landmarks(color)

        chin01 = face_landmarks_list[0]['chin'][1]
        chin15 = face_landmarks_list[0]['chin'][15]
        nose_tip = face_landmarks_list[0]['nose_tip'][2]
        left_eye = face_landmarks_list[0]['left_eye'][5]
        right_eye = face_landmarks_list[0]['right_eye'][5]

        chinDistance01 = reader.measure_depth(chin01[0], chin01[1])
        chinDistance01 = "{:.2f}".format(chinDistance01)
        chinDistance15 = reader.measure_depth(chin15[0], chin15[1])
        chinDistance15 = "{:.2f}".format(chinDistance15)
        nose_tipDistance = reader.measure_depth(nose_tip[0], nose_tip[1])
        nose_tipDistance = "{:.2f}".format(nose_tipDistance)
        left_eyeDistance = reader.measure_depth(left_eye[0], left_eye[1])
        left_eyeDistance = "{:.2f}".format(left_eyeDistance)
        right_eyeDistance = reader.measure_depth(right_eye[0], right_eye[1])
        right_eyeDistance = "{:.2f}".format(right_eyeDistance)

        if chinDistance01 != left_eyeDistance and \
            chinDistance01 != nose_tipDistance and \
            chinDistance01 != right_eyeDistance and \
            chinDistance15 != left_eyeDistance and \
            chinDistance15 != nose_tipDistance and \
            chinDistance15 != right_eyeDistance and \
            nose_tipDistance != right_eyeDistance and \
            nose_tipDistance != left_eyeDistance:
            realList.append(real)
            realImage = color
            face_location = lissilab.face_locations(realImage)
            x, y, w, h = face_location[0]
            realImage = realImage[x-20:w, h:y]

        elif chinDistance01 == left_eyeDistance and \
            chinDistance01 == nose_tipDistance and \
            chinDistance01 == right_eyeDistance and \
            chinDistance15 == left_eyeDistance and \
            chinDistance15 == nose_tipDistance and \
            chinDistance15 == right_eyeDistance and \
            nose_tipDistance == right_eyeDistance and \
            nose_tipDistance == left_eyeDistance:
            fakeList.append(fake)

    realDict = {i:realList.count(i) for i in realList}
    fakeDict = {i:fakeList.count(i) for i in fakeList}

    if realList and fakeList:
        if realDict[real] > fakeDict[fake]:
            print(real)
            cv2.imwrite(pathToAndNameOfOut, realImage)
        elif fakeDict[fake] > realDict[real]:
            print(fake)
        elif fakeDict[fake] == realDict[real]:
            print('Do another capture')
    elif realList and not fakeList:
        print(real)
        cv2.imwrite(pathToAndNameOfOut, realImage)
    elif fakeList and not realList:
        print(fake)
    
    shutil.rmtree(pathToBAG + 'Color')
    shutil.rmtree(pathToBAG + 'Depth')
    os.remove(pathToBAG + 'Color.avi')
    os.remove(pathToBAG + 'Depth.avi')
    os.remove(pathToBAG + 'meta.pkl')

In [ ]:
import json

r = open('/raid/home/labuseraber/environment/testEnvironment/dmo08082022/dpoFiles/dpoExample.json')
y = json.load(r)

ext1 = y[0]['iai_dpo_metadata'][0]['file:extension']
ext2 = y[0]['iai_dpo_metadata'][1]['file:extension']

if ext1 == 'mp4' and ext2 == 'png':
    videoName = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    imageName = y[0]['iai_dpo_metadata'][1]['id'] + '.dpo'
    print('\nvideoName: ', videoName)
    print('\nimageName: ', imageName)
elif ext2 == 'mp4' and ext1 == 'png':
    videoName = y[0]['iai_dpo_metadata'][1]['id'] + '.dpo'
    imageName = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    print('\nimageName: ', imageName)
    print('\nvideoName: ', videoName)

In [ ]:
# JSON - two .dpo

import json
import os

mainPath = "/raid/home/labuseraber/environment/testEnvironment/dmo08082022/"

r = open('dpoFiles/dpoExample.json')
r = open('dpoFiles/faceRecognitionDPO.json')
y = json.load(r)

ext1 = y[0]['iai_dpo_metadata'][0]['file:extension']
ext2 = y[0]['iai_dpo_metadata'][1]['file:extension']

if ext1 == 'mp4' or ext2 == 'mp4':
    videoName1 = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    videoName2 = y[0]['iai_dpo_metadata'][1]['id'] + '.dpo'

    if 'mp4' in ext1:
        videoName = videoName1
    elif 'mp4' in ext2:
        videoName = videoName2

    print('\nvideoName: ', videoName)
    print("base64 -d " + videoName + "> ./g.mp4")
    statusMP4 = os.system("base64 -d " + mainPath + "dpoFiles/" + videoName + "> ./g.mp4")

if ext1 == 'png' or ext2 == 'png':
    imageName1 = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    imageName2 = y[0]['iai_dpo_metadata'][1]['id'] + '.dpo'

    if 'png' in ext1:
        imageName = imageName1
    elif 'png' in ext2:
        imageName = imageName2

    print('\nimageName: ', imageName)
    print("base64 -d " + mainPath + "dpoFiles/" + imageName + "> ./r.png")
    statusPNG = os.system("base64 -d " + mainPath + "dpoFiles/" + imageName + "> ./r.png")

if ext1 == 'jpg' or ext2 == 'jpg':
    imageName1 = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    imageName2 = y[0]['iai_dpo_metadata'][1]['id'] + '.dpo'

    if 'jpg' in ext1:
        imageName = imageName1
    elif 'jpg' in ext2:
        imageName = imageName2

    print('\nimageName: ', imageName)
    print("base64 -d " + imageName + "> ./r.jpg")
    statusJPG = os.system("base64 -d " + mainPath + "dpoFiles/" + imageName + "> ./r.png")

if ext1 == 'bag' or ext2 == 'bag':
    bagName1 = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    bagName2 = y[0]['iai_dpo_metadata'][1]['id'] + '.dpo'

    if 'bag' in ext1:
        bagName = bagName1
    elif 'bag' in ext2:
        bagName = bagName2

    print('\nimageName: ', bagName)
    print("base64 -d " + bagName + "> ./a.bag")
    statusBAG = os.system("base64 -d " + mainPath + "dpoFiles/" + bagName + "> ./a.bag")

In [ ]:
# JSON - extended to three .dpo

import json
import os

r = open('/raid/home/labuseraber/environment/testEnvironment/dmo08082022/dpoFiles/dpoExampleCopy.json')
r = open('/raid/home/labuseraber/environment/testEnvironment/dmo08082022/dpoFiles/faceRecognitionDPO.json')
y = json.load(r)

ext1 = y[0]['iai_dpo_metadata'][0]['file:extension']
ext2 = y[0]['iai_dpo_metadata'][1]['file:extension']
ext3 = y[0]['iai_dpo_metadata'][2]['file:extension']

if ext1 == 'mp4' or ext2 == 'mp4' or ext3 == 'mp4':
    videoName1 = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    videoName2 = y[0]['iai_dpo_metadata'][1]['id'] + '.dpo'
    videoName3 = y[0]['iai_dpo_metadata'][2]['id'] + '.dpo'

    if 'mp4' in ext1:
        videoName = videoName1
    elif 'mp4' in ext2:
        videoName = videoName2
    elif 'mp4' in ext3:
        videoName = videoName3

    print('\nvideoName: ', videoName)
    print("base64 -d " + videoName + "> ./g" + videoName[-4])

if ext1 == 'png' or ext2 == 'png' or ext3 == 'png':
    imageName1 = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    imageName2 = y[0]['iai_dpo_metadata'][1]['id'] + '.dpo'
    imageName3 = y[0]['iai_dpo_metadata'][2]['id'] + '.dpo'

    if 'png' in ext1:
        imageName = imageName1
    elif 'png' in ext2:
        imageName = imageName2
    elif 'png' in ext3:
        imageName = imageName3

    print('\nimageName: ', imageName)
    # os.system("base64 -d " + imageName + "> ./r" + imageName[-4])
    print("base64 -d " + imageName + "> ./r" + imageName[-4])

if ext1 == 'jpg' or ext2 == 'jpg' or ext3 == 'jpg':
    imageName1 = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    imageName2 = y[0]['iai_dpo_metadata'][1]['id'] + '.dpo'
    imageName3 = y[0]['iai_dpo_metadata'][2]['id'] + '.dpo'

    if 'jpg' in ext1:
        imageName = imageName1
    elif 'jpg' in ext2:
        imageName = imageName2
    elif 'jpg' in ext3:
        imageName = imageName3

    print('\nimageName: ', imageName)
    # os.system("base64 -d " + imageName + "> ./r" + imageName[-4])
    print("base64 -d " + imageName + "> ./r" + imageName[-4])

if ext1 == 'bag' or ext2 == 'bag' or ext3 == 'bag':
    bagName1 = y[0]['iai_dpo_metadata'][0]['id'] + '.dpo'
    bagName2 = y[0]['iai_dpo_metadata'][1]['id'] + '.dpo'
    bagName3 = y[0]['iai_dpo_metadata'][2]['id'] + '.dpo'

    if 'png' in ext1:
        bagName = bagName1
    elif 'png' in ext2:
        bagName = bagName2
    elif 'png' in ext3:
        bagName = bagName3

    print('\nimageName: ', bagName)
    print("base64 -d " + bagName + "> ./a" + bagName[-4])